In [17]:
import pandas as pd
import numpy as np
from math import sqrt
import copy
from collections import Counter

In [18]:
proto_gmt = pd.read_csv("data_for_gmt.csv")
gmt = proto_gmt.T
gmt

,0,1,2,3,4,5,6,7,8,9,...,53,54,55,56,57,58,59,60,61,62
Telomere_attrition,ATM,FEN1,SIRT1,TERC,TERF1,TERF2,TERT,XRCC6,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Stem_cell_exhaustion,FOXM1,MAPK14,PLAU,STAT3,STAT5A,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Mitochondrial_dysfunction,APTX,ATP5O,CAT,COQ7,HSPD1,IGF1R,MT-CO1,POLG,RAD51,RNTL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Genomic_instability,ABL1,APEX1,BAK1,BLM,BMI1,BRCA1,BRCA2,BUB1B,CCNA2,CDK1,...,TOP1,TOP2A,TOP2B,TP53,TR,UCHL1,WRN,XPA,XRCC5,XRCC6
Epigenetic_alterations,CDK1,CREB1,DDIT3,HBP1,HDAC1,HDAC2,HMGB1,HMGB2,PML,PROP1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Deregulated_nutrient_sensing,AGPAT2,AKT1,APOE,AR,BSCL2,CACNA1A,DCY5,FGFR1,FOXO1,GH1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Cellular_senescence,ATF2,BAX,BMI1,BUB3,CREBBP,CTGF,E2F1,EGF,EP300,EPOR,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Altered_intercellular_communication,AIFM1,APOC3,AR,BDNF,CACNA1A,CEBPA,CEBPB,EGFR,EGR1,ERBB2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Loss_of_proteostasis,A2M,APP,BCL2,BSCL2,DDIT3,EEF2,GSTA4,HSPA9,MAPT,MSRA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
expression = pd.read_csv("gene_expression_complete.csv", names = ['Gene', 'Expression'], skiprows = [0])
expression.head()

,Gene,Expression
0,TPT1,14.487080
1,RPL41,14.448307
2,HLA-A,14.415246
3,TMSB4X,14.405954
4,UBA52,14.400992


Now that we have a gmt file and a list of expression values, let's confirm that we can map from one to the other. 

In [20]:
count = 0 
for i in gmt.iloc[3,:]:
    if i in set(expression.Gene):
        count += 1
        
print('So, ' + str(count) + ' of ')
# This is inelegant, as there are some NaN values, etc, but it confirms that our labels map.    
print(str(len(gmt.iloc[3,:])) + ' genes have mapped.')   

So, 58 of 
63 genes have mapped.


In [21]:
# Now we will try to do GSEA. Note that our 'expression' file is already ranked -- 
# it is a list of genes with their expression in treated relative to control, 
# ranked from highest to lowest expression

def calculate_enrichment_scores(expression, gmt):
    '''
    a function that calculates the enrichment score of a gene set in your expression data
    inputs: 
        expression data should be a list of gene ids, ordered by relative fold change between 
        treatment and control such that the gene with the greatest fold-change appears at the 
        top of the list and vice versa
        
        gmt_info should be your gmt file, parsed into a pandas DataFrame
        
    outputs:
        results, a DataFrame of gene set names and enrichment scores
        
    '''
    Enrichment_Score_dict = {}

    # for row in gmt
    for row in gmt.index:

        # find the geneset of interest; it is the intersection of the genes from the GMT file and the genes in the expression dataset
        full_gene_set = {x for x in gmt.loc[row,:] if pd.notna(x)}
        expression_set = set(expression.Gene) 
        gene_set = expression_set.intersection(full_gene_set)

        if len(gene_set) == 0:
            Enrichment_Score_dict[row] = 0
            print(str(row) + ' was 0')

        else: 
            # Now to set up our random walk penalties
            reward_amount = sqrt((len(expression) - len(gene_set))/(len(gene_set)))
            penalty_amount = -1 * sqrt(len(gene_set) / (len(expression) - len(gene_set)))
            # let's take a random walk
            expression = expression.assign(reward_or_penalty = [reward_amount if x in gene_set else penalty_amount for x in expression.Gene])
            expression = expression.assign(the_walk = np.cumsum(expression.reward_or_penalty))

            # now let's check that the random walk ends with a zero (as it should)
            if int(expression.the_walk.tail(1)) != 0:
                print(int(expression.the_walk.tail(1)))
                raise ValueError("your brownian bridge is broken")  ## just checking

            Enrichment_Score_dict[row] = round(max(expression.the_walk),2)

    results = pd.DataFrame.from_dict(Enrichment_Score_dict, orient = 'index', columns = ['Enrichment_Score'])
    return(results)

results = calculate_enrichment_scores(expression, gmt)    
print(results)

                                     Enrichment_Score
Telomere_attrition                             142.03
Stem_cell_exhaustion                            86.22
Mitochondrial_dysfunction                      222.73
Genomic_instability                            243.03
Epigenetic_alterations                         120.60
Deregulated_nutrient_sensing                   130.65
Cellular_senescence                            206.86
Altered_intercellular_communication            188.48
Loss_of_proteostasis                           257.67


Now we need to find a critical threshold value to decide what constitutes statistically significant enrichment.
We will do this empirically by generating random data and observing the fraction of times we obtain a value greater than our true values by chance alone. 

In [22]:
def emperic_p_values(expression, gmt, res, n_iterations):

    distribution = copy.copy(res)
    for i in range(n_iterations):
        pretend_ranked_list = pd.DataFrame(expression.Gene.sample(frac = 1))
        rando_scores = calculate_enrichment_scores(pretend_ranked_list, gmt)
        distribution = pd.concat([distribution, rando_scores], axis = 1)
    
    raw_p = {}
    corrected_p = {}

    # take each set of enrichment scores - the first is our result, the others occured by chance alone
    for k in distribution.index:
        list_of_values = list(distribution.loc[k,:])
        number_greater = 0
        for l in list_of_values[1:]:
            if l >= list_of_values[0]:
                number_greater += 1
        # the raw p value is the fraction of randomly generated enrichment scores that were equal to or greater than our result        
        raw_p[k] = (float(number_greater)/float(len(list_of_values[1:])))

    p_values = pd.DataFrame.from_dict(raw_p, orient = 'index', columns = ['Raw_p_value'])

    # now we can correct for multiple testing by determining an appropriate p value threshold with Bonferroni correction
    p_values = p_values.assign(p_value_threshold = 0.05 / distribution.shape[0])
    p_values = p_values.assign(Significantly_enriched = p_values.Raw_p_value <= p_values.p_value_threshold)
    return(p_values)


In [23]:
emperic_p_values(expression, gmt,results, 1000)

,Raw_p_value,p_value_threshold,Significantly_enriched
Telomere_attrition,0.064,0.005556,False
Stem_cell_exhaustion,0.348,0.005556,False
Mitochondrial_dysfunction,0.002,0.005556,True
Genomic_instability,0.002,0.005556,True
Epigenetic_alterations,0.155,0.005556,False
Deregulated_nutrient_sensing,0.118,0.005556,False
Cellular_senescence,0.003,0.005556,True
Altered_intercellular_communication,0.007,0.005556,False
Loss_of_proteostasis,0.001,0.005556,True


Now, as a comparison, let's perform GSEA using canonical KEGG pathways. I will need a bit of code to parse the KEGG file. 

In [8]:
kegg_gmt_dict = {}
with open('c2.cp.kegg.v6.2.symbols.gmt') as f:
    for line in f:
        line_bits = line.split()
        kegg_gmt_dict[line_bits[0]] = line_bits[1:]

kegg_gmt_frame = pd.DataFrame.from_dict(kegg_gmt_dict, orient='index' )
kegg_gmt = kegg_gmt_frame.iloc[:, 1:]

In [9]:
kegg_results = calculate_enrichment_scores(expression, kegg_gmt)    
print(kegg_results)

                                                    Enrichment_Score
KEGG_GLYCOLYSIS_GLUCONEOGENESIS                               210.27
KEGG_CITRATE_CYCLE_TCA_CYCLE                                  334.74
KEGG_PENTOSE_PHOSPHATE_PATHWAY                                230.16
KEGG_PENTOSE_AND_GLUCURONATE_INTERCONVERSIONS                  33.88
KEGG_FRUCTOSE_AND_MANNOSE_METABOLISM                          240.10
KEGG_GALACTOSE_METABOLISM                                     198.41
KEGG_ASCORBATE_AND_ALDARATE_METABOLISM                         28.75
KEGG_FATTY_ACID_METABOLISM                                    172.55
KEGG_STEROID_BIOSYNTHESIS                                     114.66
KEGG_PRIMARY_BILE_ACID_BIOSYNTHESIS                            47.12
KEGG_STEROID_HORMONE_BIOSYNTHESIS                              33.05
KEGG_OXIDATIVE_PHOSPHORYLATION                                572.66
KEGG_PURINE_METABOLISM                                        225.94
KEGG_PYRIMIDINE_METABOLISM        

In [16]:
kegg_p_values = emperic_p_values(expression, kegg_gmt, kegg_results, 100)
kegg_p_values[kegg_p_values.Significantly_enriched == True]

,Raw_p_value,p_value_threshold,Significantly_enriched
KEGG_GLYCOLYSIS_GLUCONEOGENESIS,0.0,0.000269,True
KEGG_CITRATE_CYCLE_TCA_CYCLE,0.0,0.000269,True
KEGG_PENTOSE_PHOSPHATE_PATHWAY,0.0,0.000269,True
KEGG_FRUCTOSE_AND_MANNOSE_METABOLISM,0.0,0.000269,True
KEGG_GALACTOSE_METABOLISM,0.0,0.000269,True
KEGG_OXIDATIVE_PHOSPHORYLATION,0.0,0.000269,True
KEGG_PURINE_METABOLISM,0.0,0.000269,True
KEGG_PYRIMIDINE_METABOLISM,0.0,0.000269,True
KEGG_VALINE_LEUCINE_AND_ISOLEUCINE_DEGRADATION,0.0,0.000269,True
KEGG_GLUTATHIONE_METABOLISM,0.0,0.000269,True
